In [135]:
import requests
from typing import Tuple, List, Dict
from itertools import chain
import json
from collections import ChainMap
from functools import reduce

In [40]:
stage_one = [5244, 5245, 5246]

meta = [
    {'editors': ['Колмаков', 'Саксонов'], 'id': 5244, 'group': 'A'},
    {'editors': ['Колмаков', 'Саксонов'], 'id': 5245, 'group': 'B'},
    {'editors': ['Колмаков', 'Саксонов'], 'id': 5246, 'group': 'C'},
    {'editors': ['Иванов', 'Кудрявцев'], 'id': 5252, 'group': 'A'},
    {'editors': ['Иванов', 'Кудрявцев'], 'id': 5253, 'group': 'B'},
    {'editors': ['Иванов', 'Кудрявцев'], 'id': 5254, 'group': 'C'},
    {'editors': ['Абрамов', 'Терентьев'], 'id': 5255, 'group': 'A'},
    {'editors': ['Абрамов', 'Терентьев'], 'id': 5256, 'group': 'B'},
    {'editors': ['Абрамов', 'Терентьев'], 'id': 5257, 'group': 'C'},
    {'editors': ['Скиренко', 'Карпук'], 'id': 5259, 'group': 'A'},
    {'editors': ['Скиренко', 'Карпук'], 'id': 5260, 'group': 'B'},
    {'editors': ['Скиренко', 'Карпук'], 'id': 5261, 'group': 'C'},
]

In [32]:
def parse_mask(mask: str) -> Tuple[int, int]:
    assert len(mask) == 30
    first_tour, second_tour = mask[:15], mask[15:]
    return first_tour.count('1'), second_tour.count('1')

In [49]:
def load_results(meta_dict: Dict) -> List:
    url = f'https://rating.chgk.info/api/tournaments/{meta_dict["id"]}/list.json'
    resp = requests.get(url).json()
    for team in resp:
        team['meta'] = meta_dict
    return resp

In [50]:
results = [load_results(dict_) for dict_ in meta]

In [64]:
flat_results = list(chain.from_iterable(results))
team_ids = set(result['idteam'] for result in flat_results)
team_dicts = [[dict_ for dict_ in flat_results if dict_['idteam'] == team_id] 
              for team_id in team_ids]

In [141]:
class Team:
    def __init__(self, dicts: List[Dict]):
        self.name = dicts[0]['base_name']
        self.id = dicts[0]['idteam']
        self.results = self.parse_results(dicts)
        self.starting_group = dicts[0]['meta']['group']
        self.rating = int(dicts[0]['tech_rating_rg'])
        
    def __repr__(self):
        return json.dumps(self.__dict__, ensure_ascii=False)
    
    @staticmethod
    def parse_results(stages: List[Dict]):
        zipped = [dict(zip(stage['meta']['editors'], parse_mask(stage['mask'])))
                   for stage in stages]
        return {**zipped[0], **zipped[1], **zipped[2], **zipped[3]}
            

In [145]:
teams = [Team(dicts) for dicts in team_dicts]

In [148]:
teams

[{"name": "Афина", "id": "2", "results": {"Колмаков": 8, "Саксонов": 10, "Иванов": 7, "Кудрявцев": 8, "Абрамов": 7, "Терентьев": 6, "Скиренко": 8, "Карпук": 5}, "starting_group": "B", "rating": 7410},
 {"name": "Зима далеко!", "id": "6432", "results": {"Колмаков": 4, "Саксонов": 5, "Иванов": 4, "Кудрявцев": 7, "Абрамов": 4, "Терентьев": 4, "Скиренко": 4, "Карпук": 3}, "starting_group": "A", "rating": 4973},
 {"name": "Тёмный лес", "id": "6790", "results": {"Колмаков": 6, "Саксонов": 4, "Иванов": 6, "Кудрявцев": 10, "Абрамов": 3, "Терентьев": 6, "Скиренко": 9, "Карпук": 7}, "starting_group": "C", "rating": 5387},
 {"name": "Команда Алхазова", "id": "3465", "results": {"Колмаков": 3, "Саксонов": 8, "Иванов": 4, "Кудрявцев": 9, "Абрамов": 5, "Терентьев": 6, "Скиренко": 7, "Карпук": 4}, "starting_group": "B", "rating": 5684},
 {"name": "Опять двойка", "id": "59456", "results": {"Колмаков": 4, "Саксонов": 6, "Иванов": 0, "Кудрявцев": 4, "Абрамов": 1, "Терентьев": 7, "Скиренко": 4, "Карпук":

In [99]:
list(zip([1, 2, 3], [4, 5, 6]))

[(1, 4), (2, 5), (3, 6)]

In [100]:
dict(_)

{1: 4, 2: 5, 3: 6}

In [105]:
a = {1: 4, 2: 5, 3: 6}
b = {234: 2323}
dict(chain(a, b))

TypeError: cannot convert dictionary update sequence element #0 to a sequence

In [116]:
dict({1: 2}, {5: 7})

TypeError: dict expected at most 1 arguments, got 2